In [2]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
#nltk.download('stopwords')
import re
import os
import codecs
from nltk.tokenize import sent_tokenize

In [3]:
TEXTDIR = "./prepro/text/"
ABSTRACTDIR = "./prepro/abstract/"
CLEANEDDIR= "./prepro/cleaned/"


In [4]:
from rouge import Rouge 

rouge = Rouge()

In [5]:
text_files=os.listdir(TEXTDIR)

In [6]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [7]:
len(word_embeddings)

400000

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    sen_new = re.sub("[^a-zA-Z]", " ",sen_new)
    return sen_new

In [9]:
# Import the kera modules
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense
from keras.models import Model


def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(100,)),
    layers.Dense(64*64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

C:\Users\jiteshnemade\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jiteshnemade\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jiteshnemade\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jiteshnemade\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               12928     
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              528384    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                262208    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 803,585
Trainable params: 803,585
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.load_weights("model_weights.h5")

In [39]:
for index in range(0,100):
    print(index)
    curr_file = codecs.open(CLEANEDDIR+text_files[index],"r","utf-8")
    curr_file_text = curr_file.read()

    curr_file = codecs.open(ABSTRACTDIR+text_files[index],"r","utf-8")
    original_abstract = curr_file.read()

    sentences = sent_tokenize(curr_file_text)
    clean_sentences = []
    for s in sentences:
        if len(s) >=10:
            clean_sentences.append(s)

    # remove stopwords from the sentences
    clean_sentences2 = clean_sentences.copy()
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]



    Y = []
    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      if type(v)==np.ndarray:
        r_score = rouge.get_scores(original_abstract,i)
        sentence_vectors.append(v)
        Y.append(r_score[0]["rouge-1"]["f"])    
    
    X_train = np.array(sentence_vectors)
    Y_train = np.array(Y)


    EPOCHS = 100
    model.fit( X_train,Y_train,epochs=EPOCHS, validation_split = 0.2)

78
Train on 110 samples, validate on 28 samples
Epoch 1/100
110/110 [==============================] - 0s 435us/sample - loss: 2.7429e-04 - mean_absolute_error: 0.0129 - mean_squared_error: 2.7429e-04 - val_loss: 1.5116e-04 - val_mean_absolute_error: 0.0106 - val_mean_squared_error: 1.5116e-04
Epoch 2/100
110/110 [==============================] - 0s 453us/sample - loss: 9.2245e-05 - mean_absolute_error: 0.0079 - mean_squared_error: 9.2245e-05 - val_loss: 1.6589e-04 - val_mean_absolute_error: 0.0109 - val_mean_squared_error: 1.6589e-04
Epoch 3/100
110/110 [==============================] - 0s 471us/sample - loss: 5.1150e-05 - mean_absolute_error: 0.0056 - mean_squared_error: 5.1150e-05 - val_loss: 1.3209e-04 - val_mean_absolute_error: 0.0092 - val_mean_squared_error: 1.3209e-04
Epoch 4/100
110/110 [==============================] - 0s 435us/sample - loss: 2.9139e-05 - mean_absolute_error: 0.0040 - mean_squared_error: 2.9139e-05 - val_loss: 1.1653e-04 - val_mean_absolute_error: 0.0082 -

Epoch 34/100
110/110 [==============================] - 0s 462us/sample - loss: 2.3306e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 2.3306e-06 - val_loss: 1.1798e-04 - val_mean_absolute_error: 0.0091 - val_mean_squared_error: 1.1798e-04
Epoch 35/100
110/110 [==============================] - 0s 508us/sample - loss: 1.4186e-06 - mean_absolute_error: 8.4332e-04 - mean_squared_error: 1.4186e-06 - val_loss: 1.4137e-04 - val_mean_absolute_error: 0.0098 - val_mean_squared_error: 1.4137e-04
Epoch 36/100
110/110 [==============================] - 0s 444us/sample - loss: 8.0488e-06 - mean_absolute_error: 0.0022 - mean_squared_error: 8.0488e-06 - val_loss: 1.9492e-04 - val_mean_absolute_error: 0.0119 - val_mean_squared_error: 1.9492e-04
Epoch 37/100
110/110 [==============================] - 0s 598us/sample - loss: 1.8213e-05 - mean_absolute_error: 0.0034 - mean_squared_error: 1.8213e-05 - val_loss: 1.4096e-04 - val_mean_absolute_error: 0.0099 - val_mean_squared_error: 1.4096e-04
Epoc

110/110 [==============================] - 0s 481us/sample - loss: 1.6417e-06 - mean_absolute_error: 8.9649e-04 - mean_squared_error: 1.6417e-06 - val_loss: 1.1357e-04 - val_mean_absolute_error: 0.0090 - val_mean_squared_error: 1.1357e-04
Epoch 68/100
110/110 [==============================] - 0s 435us/sample - loss: 3.2696e-06 - mean_absolute_error: 0.0015 - mean_squared_error: 3.2696e-06 - val_loss: 1.0216e-04 - val_mean_absolute_error: 0.0085 - val_mean_squared_error: 1.0216e-04
Epoch 69/100
110/110 [==============================] - 0s 462us/sample - loss: 4.7839e-06 - mean_absolute_error: 0.0018 - mean_squared_error: 4.7839e-06 - val_loss: 1.0100e-04 - val_mean_absolute_error: 0.0083 - val_mean_squared_error: 1.0100e-04
Epoch 70/100
110/110 [==============================] - 0s 517us/sample - loss: 1.4364e-05 - mean_absolute_error: 0.0033 - mean_squared_error: 1.4364e-05 - val_loss: 1.0387e-04 - val_mean_absolute_error: 0.0085 - val_mean_squared_error: 1.0387e-04
Epoch 71/100
110/

110/110 [==============================] - 0s 598us/sample - loss: 1.2780e-06 - mean_absolute_error: 9.2091e-04 - mean_squared_error: 1.2780e-06 - val_loss: 1.0970e-04 - val_mean_absolute_error: 0.0088 - val_mean_squared_error: 1.0970e-04
79
Train on 192 samples, validate on 49 samples
Epoch 1/100
192/192 [==============================] - 0s 493us/sample - loss: 7.2995e-04 - mean_absolute_error: 0.0190 - mean_squared_error: 7.2995e-04 - val_loss: 2.7707e-04 - val_mean_absolute_error: 0.0148 - val_mean_squared_error: 2.7707e-04
Epoch 2/100
192/192 [==============================] - 0s 519us/sample - loss: 3.9331e-04 - mean_absolute_error: 0.0140 - mean_squared_error: 3.9331e-04 - val_loss: 2.8720e-04 - val_mean_absolute_error: 0.0146 - val_mean_squared_error: 2.8720e-04
Epoch 3/100
192/192 [==============================] - 0s 545us/sample - loss: 2.3803e-04 - mean_absolute_error: 0.0111 - mean_squared_error: 2.3803e-04 - val_loss: 2.6679e-04 - val_mean_absolute_error: 0.0138 - val_mea

Epoch 33/100
192/192 [==============================] - 0s 384us/sample - loss: 2.1736e-05 - mean_absolute_error: 0.0035 - mean_squared_error: 2.1736e-05 - val_loss: 2.4710e-04 - val_mean_absolute_error: 0.0128 - val_mean_squared_error: 2.4710e-04
Epoch 34/100
192/192 [==============================] - 0s 339us/sample - loss: 1.2845e-05 - mean_absolute_error: 0.0025 - mean_squared_error: 1.2845e-05 - val_loss: 2.5009e-04 - val_mean_absolute_error: 0.0123 - val_mean_squared_error: 2.5009e-04
Epoch 35/100
192/192 [==============================] - 0s 348us/sample - loss: 1.5693e-05 - mean_absolute_error: 0.0030 - mean_squared_error: 1.5693e-05 - val_loss: 2.5171e-04 - val_mean_absolute_error: 0.0125 - val_mean_squared_error: 2.5171e-04
Epoch 36/100
192/192 [==============================] - 0s 364us/sample - loss: 1.3105e-05 - mean_absolute_error: 0.0026 - mean_squared_error: 1.3105e-05 - val_loss: 2.7578e-04 - val_mean_absolute_error: 0.0138 - val_mean_squared_error: 2.7578e-04
Epoch 37

...

258/258 [==============================] - 0s 630us/sample - loss: 1.2146e-05 - mean_absolute_error: 0.0027 - mean_squared_error: 1.2146e-05 - val_loss: 4.2174e-04 - val_mean_absolute_error: 0.0162 - val_mean_squared_error: 4.2174e-04
Epoch 54/100
258/258 [==============================] - 0s 576us/sample - loss: 6.1053e-06 - mean_absolute_error: 0.0018 - mean_squared_error: 6.1053e-06 - val_loss: 4.2792e-04 - val_mean_absolute_error: 0.0161 - val_mean_squared_error: 4.2792e-04
Epoch 55/100
258/258 [==============================] - 0s 514us/sample - loss: 5.9489e-06 - mean_absolute_error: 0.0021 - mean_squared_error: 5.9490e-06 - val_loss: 4.3014e-04 - val_mean_absolute_error: 0.0159 - val_mean_squared_error: 4.3014e-04
Epoch 56/100
258/258 [==============================] - 0s 429us/sample - loss: 1.1186e-05 - mean_absolute_error: 0.0026 - mean_squared_error: 1.1186e-05 - val_loss: 4.1059e-04 - val_mean_absolute_error: 0.0160 - val_mean_squared_error: 4.1059e-04
Epoch 57/100
258/258 

258/258 [==============================] - 0s 380us/sample - loss: 5.1521e-06 - mean_absolute_error: 0.0017 - mean_squared_error: 5.1521e-06 - val_loss: 4.2815e-04 - val_mean_absolute_error: 0.0159 - val_mean_squared_error: 4.2815e-04
Epoch 87/100
258/258 [==============================] - 0s 441us/sample - loss: 8.7222e-06 - mean_absolute_error: 0.0025 - mean_squared_error: 8.7222e-06 - val_loss: 4.5746e-04 - val_mean_absolute_error: 0.0173 - val_mean_squared_error: 4.5746e-04
Epoch 88/100
258/258 [==============================] - 0s 424us/sample - loss: 6.6603e-06 - mean_absolute_error: 0.0019 - mean_squared_error: 6.6603e-06 - val_loss: 4.3274e-04 - val_mean_absolute_error: 0.0163 - val_mean_squared_error: 4.3274e-04
Epoch 89/100
258/258 [==============================] - 0s 417us/sample - loss: 3.9089e-06 - mean_absolute_error: 0.0013 - mean_squared_error: 3.9089e-06 - val_loss: 4.1148e-04 - val_mean_absolute_error: 0.0155 - val_mean_squared_error: 4.1148e-04
Epoch 90/100
258/258 

Epoch 19/100
175/175 [==============================] - 0s 388us/sample - loss: 5.0695e-05 - mean_absolute_error: 0.0051 - mean_squared_error: 5.0695e-05 - val_loss: 3.5408e-04 - val_mean_absolute_error: 0.0138 - val_mean_squared_error: 3.5408e-04
Epoch 20/100
175/175 [==============================] - 0s 336us/sample - loss: 1.6091e-05 - mean_absolute_error: 0.0026 - mean_squared_error: 1.6091e-05 - val_loss: 3.3681e-04 - val_mean_absolute_error: 0.0138 - val_mean_squared_error: 3.3681e-04
Epoch 21/100
175/175 [==============================] - 0s 342us/sample - loss: 7.1011e-06 - mean_absolute_error: 0.0019 - mean_squared_error: 7.1011e-06 - val_loss: 3.3309e-04 - val_mean_absolute_error: 0.0135 - val_mean_squared_error: 3.3309e-04
Epoch 22/100
175/175 [==============================] - 0s 342us/sample - loss: 2.4329e-05 - mean_absolute_error: 0.0037 - mean_squared_error: 2.4329e-05 - val_loss: 3.5141e-04 - val_mean_absolute_error: 0.0141 - val_mean_squared_error: 3.5141e-04
Epoch 23

175/175 [==============================] - 0s 570us/sample - loss: 1.2590e-05 - mean_absolute_error: 0.0025 - mean_squared_error: 1.2590e-05 - val_loss: 3.5673e-04 - val_mean_absolute_error: 0.0149 - val_mean_squared_error: 3.5673e-04
Epoch 53/100
175/175 [==============================] - 0s 527us/sample - loss: 1.4949e-05 - mean_absolute_error: 0.0030 - mean_squared_error: 1.4949e-05 - val_loss: 3.0216e-04 - val_mean_absolute_error: 0.0137 - val_mean_squared_error: 3.0216e-04
Epoch 54/100
175/175 [==============================] - 0s 427us/sample - loss: 9.1024e-06 - mean_absolute_error: 0.0022 - mean_squared_error: 9.1024e-06 - val_loss: 2.6371e-04 - val_mean_absolute_error: 0.0124 - val_mean_squared_error: 2.6371e-04
Epoch 55/100
175/175 [==============================] - 0s 445us/sample - loss: 1.2298e-05 - mean_absolute_error: 0.0024 - mean_squared_error: 1.2298e-05 - val_loss: 4.1625e-04 - val_mean_absolute_error: 0.0158 - val_mean_squared_error: 4.1625e-04
Epoch 56/100
175/175 

175/175 [==============================] - 0s 393us/sample - loss: 9.5335e-06 - mean_absolute_error: 0.0024 - mean_squared_error: 9.5335e-06 - val_loss: 3.9261e-04 - val_mean_absolute_error: 0.0149 - val_mean_squared_error: 3.9261e-04
Epoch 86/100
175/175 [==============================] - 0s 376us/sample - loss: 8.6856e-06 - mean_absolute_error: 0.0023 - mean_squared_error: 8.6856e-06 - val_loss: 2.9530e-04 - val_mean_absolute_error: 0.0134 - val_mean_squared_error: 2.9530e-04
Epoch 87/100
175/175 [==============================] - 0s 393us/sample - loss: 1.3766e-05 - mean_absolute_error: 0.0029 - mean_squared_error: 1.3766e-05 - val_loss: 3.1020e-04 - val_mean_absolute_error: 0.0135 - val_mean_squared_error: 3.1020e-04
Epoch 88/100
175/175 [==============================] - 0s 382us/sample - loss: 7.0755e-06 - mean_absolute_error: 0.0019 - mean_squared_error: 7.0755e-06 - val_loss: 3.0531e-04 - val_mean_absolute_error: 0.0134 - val_mean_squared_error: 3.0531e-04
Epoch 89/100
175/175 

In [12]:
print(model.predict(X_train[:10]))
print(Y_train[:10])

[[0.04142896]
 [0.00926849]
 [0.00199416]
 [0.0429889 ]
 [0.03233917]
 [0.01497346]
 [0.02580275]
 [0.04777624]
 [0.01041963]
 [0.01551997]]
[0.02487562 0.03902439 0.         0.03921569 0.0345679  0.02487562
 0.01985112 0.01002506 0.         0.03465346]


In [42]:
model.save_weights("model_weights.h5")

In [15]:
clean_sentences2=""
original_abstract=""
avg_score=0.0
count=0
for index in range(100,120):
    curr_file = codecs.open(CLEANEDDIR+text_files[index],"r","utf-8")
    curr_file_text = curr_file.read()

    curr_file = codecs.open(ABSTRACTDIR+text_files[index],"r","utf-8")
    original_abstract = curr_file.read()

    sentences = sent_tokenize(curr_file_text)
    clean_sentences = []
    for s in sentences:
        if len(s) >=10:
            clean_sentences.append(s)

    # remove stopwords from the sentences
    clean_sentences2 = clean_sentences.copy()
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]


    Y = []

    sentence_vectors = []
    ind=0;
    for i in clean_sentences:
      
    
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
        
      if type(v)==np.ndarray:
        r_score = rouge.get_scores(original_abstract,i)
        sentence_vectors.append(v)
        Y.append(r_score[0]["rouge-1"]["f"])
      else:
        clean_sentences.pop(ind)
        clean_sentences2.pop(ind)
        ind=ind-1
      ind=ind+1

    X_train = np.array(sentence_vectors)
    Y_train = np.array(Y)
    predictions = model.predict(X_train)
    predictions=predictions.flatten()
    indices = np.argsort(predictions)
    
    abstract_selected=""
    try:
        for j in reversed(range(indices.size-10,indices.size)):
            abstract_selected=abstract_selected+clean_sentences2[j]
    except:
        continue
    else:
        score = rouge.get_scores(original_abstract,abstract_selected)
        avg_score = avg_score + score[0]["rouge-1"]["f"];
        count = count +1;
        print('This score : ',score[0]["rouge-1"]["f"],' Average score : ',avg_score/count)
        
print('The final Average Rouge score is ',avg_score/count)

This score :  0.3580470116186248  Average score :  0.3580470116186248
This score :  0.29341316945749224  Average score :  0.3257300905380585
This score :  0.2980769181952664  Average score :  0.3165123664237945
This score :  0.45238094739425866  Average score :  0.3504795116664105
This score :  0.38775509716321904  Average score :  0.3579346287657722
This score :  0.46632123888627897  Average score :  0.3759990637858566
This score :  0.48130840621506243  Average score :  0.39104325556145747
This score :  0.3790523642007202  Average score :  0.3895443941413653
This score :  0.18784529945407658  Average score :  0.3671333836205555
This score :  0.37951806728933807  Average score :  0.3683718519874337
This score :  0.3113772411543978  Average score :  0.36319052372988503
This score :  0.3975535118364522  Average score :  0.366054106072099
This score :  0.2295081920136557  Average score :  0.3555505742214495
This score :  0.22857142428801025  Average score :  0.3464806349404896
This score 

In [16]:
predictions.shape

(1048,)

In [223]:
# predictions=predictions.flatten()
# indices = np.argsort(predictions)
# print(indices)
# predictions

[315 288 305 308 420 250 306 145 174  52 254  26 120 336 292 168 286 132
 403 186 179 208 118 330 144 396 242 256  59  95 114 424 344 316 419 317
 397 244 182 357 273 129 231 253 377 148 172  96 218 139 137 125 198  38
 251 313 334 236 222 214 358 337 130 378 164 127 205  58  68 112 374  37
 225 421 345 246 414 128 223 340 171 387 237 105  33 314 255 154 277  53
 153 394   2 366  12 425  74   0 147 119 363 269   7 213 200   3 185  79
 141 155 401 418  15 300 122 239 390 350 283 191  76   5 161 373 332 386
 304 369 310 187 307 170 176  49 173 143 331 289 175 422 410 389 136 149
 431 180  44 281 282 194 252 291 293  83 166  65 262 359  86  60 181 258
 138 388 115  51 204  82 415  78 263 301 324  21  19 392 248 375 117 126
 100 247 245 121  69 196 249  56  16 399 151 265 211 405 338  72 219  94
 328 133 365 124 108  35 142 287  71  54 116 413 309  75 192  73 150  13
 311 243 296 347 169 372 238 352 104 190 297 123  70   1 183 184 134  64
 193 274 295  89 165 259 327  81 102 319 395 152 21

array([ 0.00566299,  0.01583006,  0.0052753 ,  0.00664598,  0.01727611,
        0.00819509,  0.03585684,  0.00654432,  0.02940145,  0.02199637,
        0.054896  ,  0.03161612,  0.00547197,  0.01510071,  0.026125  ,
        0.00738197,  0.01277305,  0.0166743 ,  0.01721052,  0.01200197,
        0.050055  ,  0.01192958,  0.02562154,  0.04899856,  0.04511051,
        0.04795266, -0.01177705,  0.04049151,  0.04575045,  0.02982116,
        0.0238478 ,  0.03842598,  0.01855144,  0.00463424,  0.03157482,
        0.01415507,  0.02032082,  0.00306656,  0.0005518 ,  0.03186943,
        0.03169322,  0.01832709,  0.03814555,  0.02216437,  0.01003822,
        0.01760898,  0.02304888,  0.05012043,  0.03221772,  0.009173  ,
        0.03667199,  0.0113137 , -0.01911337,  0.00515145,  0.01432181,
        0.03607459,  0.01276201,  0.01956279,  0.00277666, -0.00353309,
        0.01110021,  0.0210047 ,  0.01987746,  0.0329898 ,  0.01603837,
        0.01100355,  0.02732834,  0.01975968,  0.00284195,  0.01

In [224]:
#predictions[predictions[:,1].argsort()]
for i in reversed(range(indices.size-10,indices.size)):
    print(predictions[indices[i]])

0.061533026
0.06066887
0.059995547
0.058472887
0.054896
0.053710744
0.053154103
0.05053789
0.050368987
0.05012043


In [17]:
abstract_selected=""
for i in reversed(range(indices.size-10,indices.size)):
    abstract_selected=abstract_selected+clean_sentences2[i]

In [18]:
print(original_abstract)

recurrent neural networks (rnns) are a class of machine learning algorithms used for
applications with time-series and sequential data. recently, there has been a strong interest in executing
rnns on embedded devices. however, difficulties have arisen because rnn requires high computational
capability and a large memory space. in this paper, we review existing implementations of rnn models on
embedded platforms and discuss the methods adopted to overcome the limitations of embedded systems.
we will define the objectives of mapping rnn algorithms on embedded platforms and the challenges facing
their realization. then, we explain the components of rnn models from an implementation perspective.
we also discuss the optimizations applied to rnns to run efficiently on embedded platforms. finally, we
compare the defined objectives with the implementations and highlight some open research questions and
aspects currently not addressed for embedded rnns.
overall, applying algorithmic optimizatio

In [19]:
print(abstract_selected)

additionally  we  describe how custom embedded hardware implementations  can support flexible rnns solutions.we highlight some hardware  volume           efficient recurrent layers and algorithmic optimizations that  can enhance implementations  efficiency.finally  we deduce some opportunities  for research to fill the gap between the defined objectives and  the research work under study.a study of  the implementations in the literature shows performance high  enough for many streaming applications while also showing  a lack of flexibility.however  using analog crossbar based multipliers can achieve high performance without  relying too much on algorithmic optimizations.the second is to decrease the memory access time  for weights retrieval  which can be implemented by relying  on on chip memory for storing the weights  applying an  efficient overlap between weights loading and computations   or by computing in memory.the first is algorithmic optimizations.we conclude from the analysis

In [20]:
print(rouge.get_scores(original_abstract,abstract_selected))

[{'rouge-1': {'f': 0.4465116229546783, 'p': 0.4067796610169492, 'r': 0.4948453608247423}, 'rouge-2': {'f': 0.10280373336590558, 'p': 0.09361702127659574, 'r': 0.11398963730569948}, 'rouge-l': {'f': 0.3560606011435377, 'p': 0.31543624161073824, 'r': 0.40869565217391307}}]
